# Testing calculation from functions

In [1]:
import xarray as xr
import numpy as np
import skimage.measure as skm
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeat

from os.path import expanduser
home = expanduser("~")

import timeit

from funcs.prFuncs import *
from funcs.tasFuncs import *
from funcs.aggFuncs import *
from funcs.husFuncs import *

from funcs.vars.prVars import *
from funcs.vars.tasVars import *
from funcs.vars.husVars import *
from funcs.vars.myFuncs import *

ModuleNotFoundError: No module named 'xarray'

In [ ]:
models = [
    # 'IPSL-CM5A-MR', # 1
    'GFDL-CM3',     
    # 'GISS-E2-H',    # 3
    # 'bcc-csm1-1',   # 4
    # 'CNRM-CM5',     # 5
    # 'CCSM4',        # 6 # cannot concatanate files for rcp85 run
    # 'HadGEM2-AO',   # 7
    # 'BNU-ESM',      # 8
    # 'EC-EARTH',     # 9
    # 'FGOALS-g2',    # 10
    # 'MPI-ESM-MR',   # 11
    # 'CMCC-CM',      # 12
    # 'inmcm4',       # 13
    # 'NorESM1-M',    # 14
    # 'CanESM2',      # 15
    # 'MIROC5',       # 16
    # 'HadGEM2-CC',   # 17
    # 'MRI-CGCM3',    # 18
    # 'CESM1-BGC'     # 19
    ]
model = models[0]


experiments = [
    'historical',
    # 'rcp85'
    ]
experiment = experiments[0]       

## Precipitation extremes

In [ ]:
if switch['local_files']:
    folder = home + '/Documents/data/cmip5/' + 'ds'
    fileName = model + '_precip_' + experiment + '.nc'
    path = folder + '/' + fileName
    ds = xr.open_dataset(path)
    precip = ds.precip*60*60*24
    precip.attrs['units']= 'mm/day'

if switch['nci_files']:
    precip = get_pr(model, experiment).precip


aWeights = np.cos(precip.lat)

In [ ]:
start = timeit.default_timer()
pr_percentiles = calc_pr_percentiles(precip)
stop = timeit.default_timer()

print(pr_percentiles)
print('it takes {} seconds to calculate extreme percentiles for model: {}'.format((stop-start), model))

In [ ]:
percentile_options = [
    # 'pr95',
    # 'pr97',
    'pr99',
    # 'pr999',
    ]
percentile_option = percentile_options[0]

In [ ]:
plt.figure(figsize= (25,5))
plt.plot(pr_percentiles[percentile_option])
plt.title('high percentile precipitation rate ({}th) daily'.format(percentile_option) + ', model: ' + model + ', ' + experiment)
plt.ylabel('precipitation rate [mm/day]')
plt.xlabel('day')
plt.axhline(y=pr_percentiles[percentile_option].mean(dim='time'), color='k')

In [ ]:
start = timeit.default_timer()
rxday = calc_rxday(precip)
stop = timeit.default_timer()

print(rxday)
print('it takes {} seconds to calculate rx1day and rx5day for model: {}'.format((stop-start)/60, model))

In [ ]:
rxday_options = [
    'rx1day',
    # 'rx5day'
    ]
rxday_option = rxday_options[0]

In [ ]:
plot_snapshot(rxday[rxday_option].mean(dim='time'), 'Blues', rxday_option, model)

In [ ]:
plt.figure(figsize= (25,5))
plt.plot(rxday[rxday_option].weighted(aWeights).mean(dim=('lat','lon')))
plt.title(rxday_option + ' (annual), model: ' + model + ', ' + experiment)
plt.ylabel('precipitation rate [mm/day]')
plt.xlabel('years')
plt.axhline(y=rxday[rxday_option].weighted(aWeights).mean(dim=('time','lat','lon')), color='k')

## Organization metrics

In [ ]:
conv_threshold = pr_percentiles[percentile_option].mean(dim=('time'))

In [ ]:
start = timeit.default_timer()
numberIndex = calc_numberIndex(precip, conv_threshold)
stop = timeit.default_timer()

print(numberIndex)
print('it takes {} seconds to calculate the number index for model: {}'.format((stop-start), model))

In [ ]:
numberIndex_options = [
    'o_number',
    # 'areaf'
    ]
numberIndex_option = numberIndex_options[0]

In [ ]:
plt.figure(figsize= (25,5))
plt.plot(numberIndex[numberIndex_option])
plt.title(numberIndex_option + ' (daily), model: ' + model + ', ' + experiment)
plt.ylabel(numberIndex_option)
plt.xlabel('days')
plt.axhline(y=numberIndex[numberIndex_option].mean(dim='time'), color='k')

In [ ]:
start = timeit.default_timer()
rome = calc_rome(precip, conv_threshold)
stop = timeit.default_timer()

print('it takes {} minutes to calculate rome for model: {}'.format((stop-start)/60, model))

In [ ]:
start = timeit.default_timer()
n=8
rome_n = calc_rome_n(n, precip, conv_threshold)
stop = timeit.default_timer()

print('it takes {} minutes to calculate rome based of {} largest objects for model: {}'.format((stop-start)/60, n, model))

In [ ]:
dataset = xr.Dataset(
    data_vars = {'rome':rome, 
                'rome_n':rome_n},
    attrs = {'description': 'ROME based on all and the {} largest contiguous convective regions in the scene for each day'.format(n),
                'units':'km^2'}                  
        )
print(dataset)

In [ ]:
rome_options = [
    'rome',
    # 'rome_n'
    ]
rome_option = rome_options[0]

In [ ]:
plt.figure(figsize= (25,5))
plt.plot(dataset[rome_option])
plt.title(rome_option + ' (daily), model: ' + model + ', ' + experiment)
plt.ylabel(rome_option)
plt.xlabel('days')
plt.axhline(y=dataset[rome_option].mean(dim='time'), color='k')

In [ ]:
start = timeit.default_timer()
pwad = calc_pwad(precip, conv_threshold)
stop = timeit.default_timer()

print(pwad)
print('it takes {} seconds to calculate components of pwad for model: {}'.format((stop-start), model))

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(pwad.bins_mid,pwad.pwad)
plt.title('Precipitation Weighted Area Distribution')

## Surface temperature

In [ ]:
if switch['local_files']:
    folder = home + '/Documents/data/cmip5/' + 'ds'
    fileName = model + '_tas_' + experiment + '.nc'
    path = folder + '/' + fileName
    ds = xr.open_dataset(path)
    tas = ds.tas

if switch['nci_files']:
    tas = get_tas(model, experiment)

In [ ]:
start = timeit.default_timer()
tas_tMean = calc_tas_tMean(tas)
stop = timeit.default_timer()

print(tas_tMean)
print('it takes {} seconds to calculate tas_tMean for model: {}'.format((stop-start), model))

In [ ]:
plot_snapshot(tas_tMean['tas_tMean'], 'coolwarm', 'surface temperature', model)

In [ ]:
start = timeit.default_timer()
tas_sMean = calc_tas_sMean(tas)
stop = timeit.default_timer()

print(tas_sMean)
print('it takes {} seconds to calculate tas_sMean for model: {}'.format((stop-start), model))

In [ ]:
plt.figure(figsize= (25,5))
plt.plot(tas_sMean)
plt.title('spatial mean surface temperature (monthly), model: ' + model + ', ' + experiment)
plt.ylabel('surface temperature [deg C]')
plt.xlabel('months')
plt.axhline(y=tas_sMean.mean(dim='time'), color='k')

## Specific humidity

In [ ]:
start = timeit.default_timer()
tas_tMean = calc_hus_tMean(hus)
stop = timeit.default_timer()

print(hus_tMean)
print('it takes {} seconds to calculate hus_tMean for model: {}'.format((stop-start), model))

In [ ]:
plot_snapshot(hus_tMean['tas_tMean'].sel(plev = 1000e2), 'coolwarm', 'surface temperature', model)

In [ ]:
start = timeit.default_timer()
tas_sMean = calc_hus_sMean(hus)
stop = timeit.default_timer()

print(hus_sMean)
print('it takes {} seconds to calculate hus_sMean for model: {}'.format((stop-start), model))